In [1]:
import pandas as pd
import numpy as np

In [2]:
df_train = pd.read_csv('../../Downloads/cc_train.csv.zip')
df_test = pd.read_csv('../../Downloads/cc_test.csv')

In [3]:
df_train.head()

,LIMIT_BAL,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,BILL_AMT2,...,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,GENDER,QUALIFICATION,MARRIED,defaulter,Id
0,290000.0,50,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,NaN,199.0,...,1440.0,1017.0,3798.0,4898,5034.0,FEMALE,2,1.0,0,1
1,20000.0,49,0.0,0.0,0.0,0.0,NaN,0.0,19773.0,18459.0,...,1270.0,633.0,655.0,759,547.0,FEMALE,2,3.0,1,2
2,230000.0,28,0.0,0.0,0.0,0.0,0.0,0.0,20356.0,21504.0,...,1700.0,2007.0,1000.0,1000,1000.0,FEMALE,1,2.0,0,3
3,210000.0,31,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,8600.0,0.0,...,4231.0,4221.0,4221.0,4483,4300.0,FEMALE,1,1.0,0,4
4,300000.0,24,-2.0,-2.0,-2.0,-2.0,-1.0,-1.0,-158.0,-158.0,...,0.0,0.0,300.0,100,100.0,MALE,2,2.0,0,5


In [4]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 25 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   LIMIT_BAL      18700 non-null  float64
 1   AGE            20000 non-null  int64  
 2   PAY_0          19344 non-null  float64
 3   PAY_2          18141 non-null  float64
 4   PAY_3          18751 non-null  float64
 5   PAY_4          19316 non-null  float64
 6   PAY_5          19323 non-null  float64
 7   PAY_6          18655 non-null  float64
 8   BILL_AMT1      18130 non-null  float64
 9   BILL_AMT2      18681 non-null  float64
 10  BILL_AMT3      19323 non-null  float64
 11  BILL_AMT4      20000 non-null  int64  
 12  BILL_AMT5      19318 non-null  float64
 13  BILL_AMT6      20000 non-null  int64  
 14  PAY_AMT1       18690 non-null  float64
 15  PAY_AMT2       18763 non-null  float64
 16  PAY_AMT3       18691 non-null  float64
 17  PAY_AMT4       19318 non-null  float64
 18  PAY_AM

In [5]:
df_train.isna().sum()

LIMIT_BAL        1300
AGE                 0
PAY_0             656
PAY_2            1859
PAY_3            1249
PAY_4             684
PAY_5             677
PAY_6            1345
BILL_AMT1        1870
BILL_AMT2        1319
BILL_AMT3         677
BILL_AMT4           0
BILL_AMT5         682
BILL_AMT6           0
PAY_AMT1         1310
PAY_AMT2         1237
PAY_AMT3         1309
PAY_AMT4          682
PAY_AMT5            0
PAY_AMT6          688
GENDER              0
QUALIFICATION       0
MARRIED          1326
defaulter           0
Id                  0
dtype: int64

In [6]:
columns_numeric = list(df_train.select_dtypes(exclude='object').columns)
columns_numeric.remove('defaulter')
columns_numeric.remove('Id')

#columns_numeric = list(df_test.select_dtypes(exclude='object').columns)

In [7]:
columns_numeric

['LIMIT_BAL',
 'AGE',
 'PAY_0',
 'PAY_2',
 'PAY_3',
 'PAY_4',
 'PAY_5',
 'PAY_6',
 'BILL_AMT1',
 'BILL_AMT2',
 'BILL_AMT3',
 'BILL_AMT4',
 'BILL_AMT5',
 'BILL_AMT6',
 'PAY_AMT1',
 'PAY_AMT2',
 'PAY_AMT3',
 'PAY_AMT4',
 'PAY_AMT5',
 'PAY_AMT6',
 'QUALIFICATION',
 'MARRIED']

In [8]:
# Imputation
from sklearn.impute import SimpleImputer

imputer_num = SimpleImputer(strategy='mean')
imputer_num.fit(df_train[columns_numeric])
df_train[columns_numeric] = imputer_num.transform(df_train[columns_numeric])

df_test[columns_numeric] = imputer_num.transform(df_test[columns_numeric])


In [9]:
columns_categorical = list(df_train.select_dtypes(include='object').columns)

imputer_cat = SimpleImputer(strategy='most_frequent')
imputer_cat.fit(df_train[columns_categorical])
df_train[columns_categorical] = imputer_cat.transform(df_train[columns_categorical])

df_test[columns_categorical] = imputer_cat.transform(df_test[columns_categorical])



In [10]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(handle_unknown='ignore')
ohe.fit(df_train[columns_categorical])
temp = pd.DataFrame(data=ohe.transform(df_train[columns_categorical]).toarray(), columns=ohe.get_feature_names())

columns_final = columns_numeric + list(ohe.get_feature_names())

df_train = pd.concat([df_train, temp], axis=1)


temp = pd.DataFrame(data=ohe.transform(df_test[columns_categorical]).toarray(), columns=ohe.get_feature_names())

df_test = pd.concat([df_test, temp], axis=1)

In [11]:
#scaling

from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaler.fit(df_train[columns_numeric])
df_train[columns_numeric] = scaler.transform(df_train[columns_numeric])

df_test[columns_numeric] = scaler.transform(df_test[columns_numeric])



In [12]:
columns_final

['LIMIT_BAL',
 'AGE',
 'PAY_0',
 'PAY_2',
 'PAY_3',
 'PAY_4',
 'PAY_5',
 'PAY_6',
 'BILL_AMT1',
 'BILL_AMT2',
 'BILL_AMT3',
 'BILL_AMT4',
 'BILL_AMT5',
 'BILL_AMT6',
 'PAY_AMT1',
 'PAY_AMT2',
 'PAY_AMT3',
 'PAY_AMT4',
 'PAY_AMT5',
 'PAY_AMT6',
 'QUALIFICATION',
 'MARRIED',
 'x0_FEMALE',
 'x0_MALE']

In [13]:
df_test

,LIMIT_BAL,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,BILL_AMT2,...,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,GENDER,QUALIFICATION,MARRIED,Id,x0_FEMALE,x0_MALE
0,0.343434,0.232143,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.016594,0.066791,...,0.000294,0.001702,0.002980,0.010131,FEMALE,0.166667,0.666667,20001,1.0,0.0
1,0.000000,0.160714,0.2,0.222222,0.200000,0.200000,0.200000,0.2,0.024714,0.072483,...,0.001116,0.001908,0.002345,0.003023,MALE,0.166667,0.666667,20002,0.0,1.0
2,0.343434,0.285714,0.2,0.222222,0.200000,0.200000,0.200000,0.2,0.109655,0.180427,...,0.005580,0.009100,0.009378,0.009458,MALE,0.166667,0.666667,20003,0.0,1.0
3,0.010101,0.392857,0.1,0.111111,0.100000,0.000000,0.100000,0.1,0.017588,0.066695,...,0.000000,0.012025,0.026305,0.000000,FEMALE,0.166667,0.333333,20004,1.0,0.0
4,0.202020,0.375000,0.3,0.111111,0.183142,0.100000,0.100000,0.1,0.015623,0.066837,...,0.000363,0.000614,0.000762,0.000615,MALE,0.333333,0.333333,20005,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,0.333333,0.232143,0.2,0.206843,0.200000,0.200000,0.200000,0.2,0.210992,0.246858,...,0.006696,0.011344,0.014067,0.009458,MALE,0.166667,0.666667,24996,0.0,1.0
4996,0.494949,0.196429,0.1,0.111111,0.100000,0.177858,0.200000,0.2,0.032522,0.076658,...,0.010350,0.007563,0.011723,0.101989,FEMALE,0.166667,0.666667,24997,1.0,0.0
4997,0.191919,0.410714,0.4,0.444444,0.400000,0.400000,0.173074,0.4,0.187378,0.229454,...,0.007812,0.012479,0.016412,0.013430,MALE,0.166667,0.333333,24998,0.0,1.0
4998,0.343434,0.321429,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.021936,0.092623,...,0.037969,0.037815,0.275831,0.140138,FEMALE,0.166667,0.333333,24999,1.0,0.0


In [14]:
df_train.corr()['defaulter']

LIMIT_BAL       -0.148857
AGE              0.006791
PAY_0            0.314856
PAY_2            0.249364
PAY_3            0.226213
PAY_4            0.211096
PAY_5            0.199007
PAY_6            0.176515
BILL_AMT1       -0.023935
BILL_AMT2       -0.025985
BILL_AMT3       -0.022905
BILL_AMT4       -0.020788
BILL_AMT5       -0.019400
BILL_AMT6       -0.018142
PAY_AMT1        -0.074846
PAY_AMT2        -0.058260
PAY_AMT3        -0.052440
PAY_AMT4        -0.061143
PAY_AMT5        -0.055519
PAY_AMT6        -0.056235
QUALIFICATION    0.030473
MARRIED         -0.023324
defaulter        1.000000
Id               0.006318
x0_FEMALE       -0.043314
x0_MALE          0.043314
Name: defaulter, dtype: float64

In [15]:
X = df_train[columns_final]
y = df_train['defaulter']

# model building

In [16]:
# LogReg

from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model.fit(X,y)

C:\Users\kulas\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [17]:
model.score(X,y)

0.80495

In [18]:
#KNN

from sklearn.neighbors import KNeighborsClassifier

model = KNeighborsClassifier(n_neighbors=5)
model.fit(X,y)

KNeighborsClassifier()

In [19]:
model.score(X,y)

0.8409

In [20]:
yp = model.predict(df_test[columns_final])
yp


array([0, 0, 0, ..., 1, 0, 0], dtype=int64)

In [21]:
df_submission = pd.DataFrame({'Id':df_test['Id'], 'defaulter':yp})

In [22]:
df_submission.to_csv('KNN_model.csv', index=False)

In [23]:
#DT

from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier(max_depth=5, min_samples_leaf=4)
model.fit(X, y)

DecisionTreeClassifier(max_depth=5, min_samples_leaf=4)

In [24]:
depths = [1,2,3,4,5]
samples = [1,2,3,4]
for d in depths:
    for s in samples:
        model = DecisionTreeClassifier(max_depth=d, min_samples_leaf=s)
        model.fit(X,y)
        train_score = round(model.score(X,y),2)
        print('Depth:', d,'Samples at leaf:', s, 'Train Acc:',train_score)

Depth: 1 Samples at leaf: 1 Train Acc: 0.82
Depth: 1 Samples at leaf: 2 Train Acc: 0.82
Depth: 1 Samples at leaf: 3 Train Acc: 0.82
Depth: 1 Samples at leaf: 4 Train Acc: 0.82
Depth: 2 Samples at leaf: 1 Train Acc: 0.82
Depth: 2 Samples at leaf: 2 Train Acc: 0.82
Depth: 2 Samples at leaf: 3 Train Acc: 0.82
Depth: 2 Samples at leaf: 4 Train Acc: 0.82
Depth: 3 Samples at leaf: 1 Train Acc: 0.82
Depth: 3 Samples at leaf: 2 Train Acc: 0.82
Depth: 3 Samples at leaf: 3 Train Acc: 0.82
Depth: 3 Samples at leaf: 4 Train Acc: 0.82
Depth: 4 Samples at leaf: 1 Train Acc: 0.82
Depth: 4 Samples at leaf: 2 Train Acc: 0.82
Depth: 4 Samples at leaf: 3 Train Acc: 0.82
Depth: 4 Samples at leaf: 4 Train Acc: 0.82
Depth: 5 Samples at leaf: 1 Train Acc: 0.82
Depth: 5 Samples at leaf: 2 Train Acc: 0.82
Depth: 5 Samples at leaf: 3 Train Acc: 0.82
Depth: 5 Samples at leaf: 4 Train Acc: 0.82


In [25]:
model.score(X,y)

0.8228

In [26]:
#RF

from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators=200)
model.fit(X,y)

RandomForestClassifier(n_estimators=200)

In [27]:
model.score(X,y)

0.99995

In [28]:
df_submission = pd.DataFrame({'Id':df_test['Id'], 'defaulter':yp})
df_submission.to_csv('RF_model.csv', index=False)